# ************************************************************* #
#               Copyright (C) 2022 Jorge Brenes Alfaro.
#               EL5617 Trabajo Final de Graduación.
#               Escuela de Ingeniería Electrónica.
#               Tecnológico de Costa Rica.
# ************************************************************* #

### Se obtiene las versión de cuda

In [ ]:
!nvidia-smi
!nvcc -V
!gcc --version

## Instalación de WANDB y Pytorch

In [ ]:
!pip install wandb
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

## Libraries

In [ ]:
import os
import wandb
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt
from wandb.keras import WandbCallback

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
wandb.login()

## Si se hace uso de google colab

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
root = '/content/drive/Othercomputers/Mi portátil/TEC/TFG/Datos_Recolectados/'

In [ ]:
wandb.config = {
    "epochs": 10,
    "batch_size": 100,
    "learning_rate":0.001,
    "Dropout": 0.35,
    "n_layers":1
}

## Procesamiento de datos

In [ ]:
#Directorio de la CPU
# root = '/Users/jorge/Documents/TEC/TFG/Datos_Recolectados/'

Dir = os.listdir(root)
pwm = np.array([])
angle = np.array([])

# Read all the .csv files and make an nx4 array
# Next, separate the pwm value and angle in their respective arrays.
print('******************* Process the Dataset *******************',flush=True)
print('Recolecting Data',flush=True)
for filename in Dir:
    files = pd.read_csv(root + filename)
    pwm = np.append(pwm, np.concatenate((np.zeros(100),files.values[:,2])))
    angle = np.append(angle, np.concatenate((np.zeros(100),files.values[:,3])))

X_train = []
Y_train = []
window = wandb.config['batch_size']

#For each element of training set, we have "window" previous training set elements
print('Accommodating data for the GRU network',flush=True)
for i in range(window,pwm.shape[0]):
    X_train.append(pwm[i-window:i])
    Y_train.append(angle[i])
X_train, Y_train = np.array(X_train), np.array(Y_train) # Input and output arrays

# Separate the values in train, validation and test data/label
train_data, val_data, test_data = [],[],[]
train_label, val_label, test_label = [],[],[]
train_lenght = int(len(X_train)*3/5)
val_lenght = int(len(X_train)*4/5)

# Use 3/5 of the total data set for training 
# and 1/5 for validation and testing.
print('Separating data in training, validation and testing',flush=True)
for i,j in zip(X_train[:train_lenght],Y_train[:train_lenght]):
    train_data.append(i)
    train_label.append(j)

for i,j in zip(X_train[train_lenght:val_lenght],Y_train[train_lenght:val_lenght]):
    val_data.append(i)
    val_label.append(j)
    
for i,j in zip(X_train[val_lenght:],Y_train[val_lenght:]):
    test_data.append(i)
    test_label.append(j)
    
train_data, val_data, test_data = np.array(train_data), np.array(val_data), np.array(test_data)
train_label, val_label, test_label = torch.from_numpy(np.array(train_label)), torch.from_numpy(np.array(val_label)), torch.from_numpy(np.array(test_label))

print('Total train data is: ', len(train_data), flush=True)
print('Total validation data is: ', len(val_data), flush=True)
print('Total test data is: ', len(test_data), flush=True)

# Reshape the arrays (n,window,1). Where n is the total amount of data in the array
print('Reshape arrays to tensors',flush=True)
train_data = torch.from_numpy(np.reshape(train_data,(train_data.shape[0],train_data.shape[1],1)))
val_data = torch.from_numpy(np.reshape(val_data,(val_data.shape[0],val_data.shape[1],1)))
test_data = torch.from_numpy(np.reshape(test_data,(test_data.shape[0],test_data.shape[1],1)))
print('******************* Finish *******************',flush=True)

## Definición de tamaños

In [ ]:
sequence_length = train_data.shape[0]
hidden_size = train_data.shape[1]
num_layers = train_data.shape[2]
output_size = train_label.shape[0]
print(sequence_length,hidden_size,num_layers,output_size)

# Red Neuronal Recurrente GRU

In [ ]:
class GRUNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers, output_dim, drop_prob=0.2):
        super(GRUNetwork, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_size
        
        self.gru = nn.GRU(input_size, hidden_size, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_size, output_dim)
        #self.relu = nn.ReLU()
        
    def forward(self, x):
        h0 = torch.zeros(self.output_dim,x.size(0),self.hidden_size).to(device)
        out, _ = self.gru(x, h0)
        out = out[:, -1, :]
        out = self.fc(out[:,-1,:])
        #out = self.fc(self.relu(out[:,-1]))
        return out

## Modelo

In [ ]:
model = GRUNetwork(sequence_length, hidden_size, num_layers, output_size, wandb.config['Dropout']).to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=wandb.config['learning_rate'])
hystory = pd.DataFrame()

## Entrenamiento del modelo

In [ ]:
for epoch in range(1, wandb.config["epochs"] + 1):
    y_train = model(train_data.to(device))
    loss = loss_function(input=y_train, target=train_label)
    loss.backward() # Backward propagation. Calculate all the gradients needed to adjust the weights.
    optimizer.step() # Uses gradients to change weight values
    #optimizer.zero_grad() # Don't accumulate gradients in epoch iterations.
    
    # Calculate the accuracy without modifying the weights of the neural network
    with torch.no_grad():
        y_train = model(train_data)
        correct = (y_train == train_label).sum()
        accuracy_train = 100*correct/float(len(test_data))
    
    df_temp = pd.DataFrame(data={
        'Epoch': epoch,
        'Loss': round(loss.item(),5),
        'Accuracy': round(accuracy.item(),5)},index=[0])